In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import torch
import torch.nn as nn
import torch.nn.functional as F
import  numpy as np
import pandas as pd
import gensim

In [2]:
with open('pride_and_prejudice.txt', 'r') as f:
    text = f.read()
    

In [3]:
def getsentences(text):
    sentence_list = []
    sentences = text.split('\n')
    for i in sentences:
        sentence = i.split('.')
        for u in sentence:
            sentence_list.append(u)
    return sentence_list
        

In [4]:
sentences = getsentences(text)

In [5]:
sentences = [x for x in sentences if x != '' ]

In [6]:
text_list = [x for x in gensim.utils.tokenize(text)]

In [7]:
text_list = [gensim.parsing.remove_stopwords(x) for x in text_list]
text_list = [x for x in text_list if x != '' ]

In [8]:
len(text_list)

54762

In [9]:
vectorizer = CountVectorizer()
vectorizer.fit(text_list)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [10]:
def getwordpairs(sentences):
    word_pairs =[]
    for x in sentences:
        sentence_tokens = [y for y in gensim.utils.tokenize(x)]
        sentence_tokens = [gensim.parsing.remove_stopwords(i) for i in sentence_tokens]
        sentence_tokens = [i for i in sentence_tokens if i != '' ]
        for i in range(len(sentence_tokens)):
            target = sentence_tokens[i]
            target = vectorizer.transform([target])[0]
            
            index = torch.LongTensor([[0],[np.argmax(target.toarray())]])
            v = torch.FloatTensor([target.toarray()[0][np.argmax(target.toarray())]])
            target = torch.sparse.FloatTensor(index,v, torch.Size([1, 6518]))
            
            if i + 1 <= (len(sentence_tokens) -1):
                predictor = sentence_tokens[i +1]
                predictor = vectorizer.transform([predictor])[0]
                
                index = torch.LongTensor([[0],
                     [np.argmax(predictor.toarray())]])
                v = torch.FloatTensor([predictor.toarray()[0][np.argmax(predictor.toarray())]])
                predictor = torch.sparse.FloatTensor(index,v, torch.Size([1, 6518]))
                
                word_pairs.append([target,predictor])
            if i + 2 <= (len(sentence_tokens) -1):
                predictor2 = sentence_tokens[i +2]
                predictor2 = vectorizer.transform([predictor2])[0]
                
                index = torch.LongTensor([[0],
                     [np.argmax(predictor2.toarray())]])
                v = torch.FloatTensor([predictor2.toarray()[0][np.argmax(predictor2.toarray())]])
                predictor2 = torch.sparse.FloatTensor(index,v, torch.Size([1, 6518]))
                
                word_pairs.append([target,predictor2])
                
            if i - 1 >= 0:
                predictor3 = sentence_tokens[i -1]
                predictor3 = vectorizer.transform([predictor3])[0]
                
                index = torch.LongTensor([[0],[np.argmax(predictor3.toarray())]])
                v = torch.FloatTensor([predictor3.toarray()[0][np.argmax(predictor3.toarray())]])
                predictor3 = torch.sparse.FloatTensor(index,v, torch.Size([1, 6518]))
                
                word_pairs.append([target,predictor3])
            if i - 2 >= 0:
                predictor4 = sentence_tokens[i -2]
                predictor4 = vectorizer.transform([predictor4])[0]
                
                index = torch.LongTensor([[0],[np.argmax(predictor4.toarray())]])
                v = torch.FloatTensor([predictor4.toarray()[0][np.argmax(predictor4.toarray())]])
                predictor4 = torch.sparse.FloatTensor(index,v, torch.Size([1, 6518]))
                
                word_pairs.append([target,predictor4])
    return word_pairs

In [11]:
word_pairs = getwordpairs(sentences)

In [12]:
len(word_pairs)

127432

In [13]:
# word_pairs = np.array(word_pairs)

In [14]:
X = [x[0] for x in word_pairs]
y = [x[1] for x in word_pairs]

In [15]:
X[0].shape

torch.Size([1, 6518])

In [16]:
newX = torch.cat(X, dim=0)
newy = torch.cat(y, dim=0)

In [17]:
newX.shape

torch.Size([127432, 6518])

In [18]:
newy.shape

torch.Size([127432, 6518])

In [19]:
dims_of_onehot_words = X[0].shape[1]
word_embedding_size = 5

In [20]:
import pickle

In [21]:
# with open('word_pairs.pickle', 'wb') as f:
#     pickle.dump(word_pairs, f)

In [22]:
class word2vec(torch.nn.Module):
    def __init__(self):
        super(word2vec, self).__init__()
        self.dense1 = nn.Linear(in_features=dims_of_onehot_words, out_features=word_embedding_size)
        self.dense2 = nn.Linear(in_features=word_embedding_size, out_features=dims_of_onehot_words)
    
    def forward(self, t):
        t = self.dense1(t)
        t = self.dense2(t)
#         t = F.softmax(t, dim=1)
        return t

In [23]:
# predictions = network(newX)

In [24]:
# loss = F.cross_entropy(predictions, torch.argmax(newy.to_dense(), axis = 1))

In [25]:
# loss

In [26]:
# predictions.shape

In [27]:
batch_size = 100

In [28]:
import math

In [29]:
network = word2vec().cuda()

In [30]:


optimizer = torch.optim.Adam(network.parameters())
epochs = 100

for epoch in range(epochs):
    current_batch = 1
    total_loss = 0
    correct = 0
    total = 0
    for batch in range(math.ceil(len(X)/batch_size)):
        batch_X = X[batch*batch_size:current_batch*batch_size]
        batch_X = torch.cat(batch_X, dim =0)
        batch_X =batch_X.cuda()
#         print(batch_X.shape)
        batch_y = y[batch*batch_size:current_batch*batch_size]
        batch_y = torch.cat(batch_y, dim =0)
        batch_y = batch_y.cuda()

        predictions = network(batch_X)
        predictions = predictions
        
        predictions_argmax = torch.argmax(batch_y.to_dense(),axis=1)
        loss = F.cross_entropy(predictions,predictions_argmax)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        current_batch +=1
        total_loss += loss.item()
        
        correct += torch.sum(torch.argmax(predictions, axis= 1) == predictions_argmax).item()
        total += batch_size
        
        
#         del(loss)
#         del(predictions)
#         del(batch_X)
#         del(batch_y)

        
    print(epoch, 'loss: ',total_loss, 'accuracy :', correct/total)

0 loss:  10228.992059707642 accuracy : 0.04442352941176471
1 loss:  9501.765815258026 accuracy : 0.0568156862745098
2 loss:  9441.246807575226 accuracy : 0.0568156862745098
3 loss:  9409.754173278809 accuracy : 0.0568156862745098
4 loss:  9388.327414989471 accuracy : 0.0568156862745098
5 loss:  9371.45474243164 accuracy : 0.05698039215686274
6 loss:  9356.178217887878 accuracy : 0.05733333333333333
7 loss:  9340.156388759613 accuracy : 0.05776470588235294
8 loss:  9322.360359191895 accuracy : 0.05818039215686274
9 loss:  9303.779684066772 accuracy : 0.05833725490196078
10 loss:  9285.641788959503 accuracy : 0.058635294117647056
11 loss:  9267.763911247253 accuracy : 0.05923921568627451
12 loss:  9250.272605419159 accuracy : 0.05979607843137255
13 loss:  9233.07406949997 accuracy : 0.0600078431372549
14 loss:  9216.048055171967 accuracy : 0.06028235294117647
15 loss:  9199.273733139038 accuracy : 0.0607921568627451
16 loss:  9182.750528335571 accuracy : 0.06149803921568627
17 loss:  916

In [31]:
word_embedding = network.dense1.weight.cpu().detach().numpy()

In [32]:
with open('word_embedding.pickle', 'wb') as f:
    pickle.dump(word_embedding, f)

In [33]:
vectorizer.vocabulary_

{'the': 5849,
 'project': 4578,
 'gutenberg': 2790,
 'ebook': 2002,
 'pride': 4523,
 'prejudice': 4469,
 'jane': 3354,
 'austen': 621,
 'this': 5866,
 'use': 6184,
 'cost': 1415,
 'restrictions': 5013,
 'whatsoever': 6375,
 'you': 6508,
 'copy': 1396,
 'away': 645,
 'terms': 5834,
 'license': 3522,
 'included': 3098,
 'online': 4069,
 'www': 6494,
 'org': 4095,
 'title': 5904,
 'author': 623,
 'release': 4863,
 'date': 1521,
 'august': 617,
 'last': 3460,
 'updated': 6175,
 'november': 3976,
 'language': 3453,
 'english': 2111,
 'character': 997,
 'set': 5273,
 'encoding': 2080,
 'utf': 6192,
 'start': 5550,
 'of': 4039,
 'and': 403,
 'produced': 4557,
 'anonymous': 427,
 'volunteers': 6282,
 'david': 1526,
 'widger': 6401,
 'there': 5855,
 'is': 3349,
 'an': 401,
 'illustrated': 3008,
 'edition': 2012,
 'which': 6381,
 'may': 3689,
 'viewed': 6248,
 'at': 577,
 'cover': 1448,
 'by': 898,
 'contents': 1347,
 'chapter': 996,
 'it': 3350,
 'truth': 5992,
 'universally': 6111,
 'acknowled

In [34]:
he = vectorizer.transform(['he'])[0]
index = torch.LongTensor([[0],[np.argmax(he.toarray())]])
v = torch.FloatTensor([he.toarray()[0][np.argmax(he.toarray())]])
he = torch.sparse.FloatTensor(index,v, torch.Size([1, 6518]))

In [35]:
she = vectorizer.transform(['she'])[0]
index = torch.LongTensor([[0],[np.argmax(she.toarray())]])
v = torch.FloatTensor([she.toarray()[0][np.argmax(she.toarray())]])
she = torch.sparse.FloatTensor(index,v, torch.Size([1, 6518]))

In [44]:
talking = vectorizer.transform(['talking'])[0]
index = torch.LongTensor([[0],[np.argmax(talking.toarray())]])
v = torch.FloatTensor([talking.toarray()[0][np.argmax(talking.toarray())]])
talking = torch.sparse.FloatTensor(index,v, torch.Size([1, 6518]))

In [48]:
speaking = vectorizer.transform(['speaking'])[0]
index = torch.LongTensor([[0],[np.argmax(speaking.toarray())]])
v = torch.FloatTensor([speaking.toarray()[0][np.argmax(speaking.toarray())]])
speaking = torch.sparse.FloatTensor(index,v, torch.Size([1, 6518]))

In [36]:
torch.sparse.mm(he, torch.tensor(word_embedding).T)

tensor([[-0.6084, -0.5063,  0.4280,  0.2836, -0.0397]])

In [38]:
torch.sparse.mm(she, torch.tensor(word_embedding).T)

tensor([[-0.7989, -0.4558,  0.5751, -0.2369, -0.5190]])

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
cosine_similarity(torch.sparse.mm(he, torch.tensor(word_embedding).T),torch.sparse.mm(she, torch.tensor(word_embedding).T))

array([[0.79176676]], dtype=float32)

In [47]:
cosine_similarity(torch.sparse.mm(she, torch.tensor(word_embedding).T),torch.sparse.mm(talking, torch.tensor(word_embedding).T))

array([[0.7153545]], dtype=float32)

In [49]:
cosine_similarity(torch.sparse.mm(speaking, torch.tensor(word_embedding).T),torch.sparse.mm(talking, torch.tensor(word_embedding).T))

array([[-0.3152054]], dtype=float32)